# Part 1

In [37]:
### Import libaries
import pandas as pd
import numpy as np
import requests

In [43]:
### Pull in Wiki web page
get_wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(get_wiki)

### Create data drame
df = pd.read_html(req.content, header=0)[0]

### Remove boroughs listed as 'Not assigned'
df = df[df.Borough != 'Not assigned']


In [45]:
### Remove neighborhoods listed as 'Not assigned'
df.Neighborhood.replace('Not assigned', df.Borough, inplace=True)


In [47]:
### Group neighborhoods by 'Postal Code'
df=df.groupby("Postal Code").agg(lambda x:','.join(set(x)))

df.head()

,Borough,Neighborhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [48]:
df.shape

(103, 2)

# Part 2

In [49]:
### Import lat / lon data
lat_lon=pd.read_csv("http://cocl.us/Geospatial_data")
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [51]:
### Add latitude and logitude values to original data frame
df['Latitude']=lat_lon['Latitude'].values
df['Longitude']=lat_lon['Longitude'].values

df.head()

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3

In [54]:
### Import libraries for clustering and mapping
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

In [64]:
from geopy.geocoders import Nominatim

In [85]:
### Limit data to just toronto
toronto_data = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
neighborhoods = toronto_data

address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [86]:
### Create map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# Add neighborhood markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [84]:
### Define foursquare credentials
CLIENT_ID = 'HTGZRSO2TRM1YIWPCVYQEYSCJ42QZXCELI2HP4QNXVJQRA4F' # your Foursquare ID
CLIENT_SECRET = 'F2PBYRHYFSZGRS5FNH42OS5AIJJ3T254HM4SUVGADLKJDGYH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [87]:
### Explore first neighborhood
toronto_data.loc[0, 'Neighborhood']

'The Beaches'

In [88]:
### Get that neighborhoods lat and lon
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [111]:
### Get top 100 venues from withon 500 m of here
LIMIT = 100
radius = 1000

### Create the request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=HTGZRSO2TRM1YIWPCVYQEYSCJ42QZXCELI2HP4QNXVJQRA4F&client_secret=F2PBYRHYFSZGRS5FNH42OS5AIJJ3T254HM4SUVGADLKJDGYH&v=20180605&ll=43.67635739999999,-79.2930312&radius=1000&limit=100'

In [112]:
### Import JSON library
from pandas.io.json import json_normalize

In [113]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f077e3a3638df69a2a29ded'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 72,
  'suggestedBounds': {'ne': {'lat': 43.685357409000005,
    'lng': -79.28061062898105},
   'sw': {'lat': 43.66735739099998, 'lng': -79.30545177101895}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'lab

In [117]:
### Pull in the categories of our top 100 venues
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [125]:
### Clean json and structure it as a new data frame
venues = results['response']['groups'][0]['items']
    
top100_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
top100_venues =top100_venues.loc[:, filtered_columns]

# filter the category for each row
top100_venues['venue.categories'] = top100_venues.apply(get_category_type, axis=1)

# clean columns
top100_venues.columns = [col.split(".")[-1] for col in top100_venues.columns]

top100_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
2,The Beech Tree,Gastropub,43.680493,-79.288846
3,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993
4,Beaches Bake Shop,Bakery,43.680363,-79.289692


In [126]:
### Count venues to confirm
print('{} venues were returned by Foursquare.'.format(top100_venues.shape[0]))

72 venues were returned by Foursquare.


In [127]:
### Create function to repeat for all neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [128]:
### Run for all neighborhoods
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [129]:
### Look at new data frame
print(toronto_venues.shape)
toronto_venues.head()

(3179, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
3,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
4,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery


In [130]:
### Count venues in each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",47,47,47,47,47,47
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100


In [131]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


### Explore Neighborhoods

In [132]:
### One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [133]:
### Group by neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.020000,...,0.00,0.00,0.010000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.01,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.00,0.00,0.000000,0.021277,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.00,0.066667,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
4,Central Bay Street,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.020000,...,0.00,0.01,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.020000
5,Christie,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.020000,0.000000,0.00,0.010000,0.00,0.010000,0.000000,0.000000
6,Church and Wellesley,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.000000,0.000000,0.01,0.000000,0.00,0.000000,0.000000,0.020000
7,"Commerce Court, Victoria Hotel",0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.020000,...,0.00,0.00,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
8,Davisville,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.010000,0.010000,0.00,0.010000,0.00,0.010000,0.000000,0.010000
9,Davisville North,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.010000,0.010000,0.00,0.010000,0.00,0.010000,0.000000,0.020000


In [154]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Park,Beer Bar,Bakery,Farmers Market,Seafood Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Restaurant,Bar,Bakery,Furniture / Home Store,Gift Shop,Tibetan Restaurant,Thrift / Vintage Store,Performing Arts Venue
2,"Business reply mail Processing Centre, South C...",Park,Pizza Place,Brewery,Coffee Shop,Fast Food Restaurant,Burrito Place,Sushi Restaurant,Bakery,Italian Restaurant,Electronics Store
3,"CN Tower, King and Spadina, Railway Lands, Har...",Café,Harbor / Marina,Coffee Shop,Garden,Airport,Sculpture Garden,Dog Run,Sushi Restaurant,Scenic Lookout,Park
4,Central Bay Street,Coffee Shop,Clothing Store,Ramen Restaurant,Park,Café,Gastropub,Furniture / Home Store,Sushi Restaurant,Japanese Restaurant,Juice Bar


In [155]:
### Create function for most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Cluster Neighborhoods

In [157]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 4, 0, 0, 0, 2, 0, 0], dtype=int32)

In [158]:
### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,Berczy Park,43.676357,-79.293031,2,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Park,Beer Bar,Bakery,Farmers Market,Seafood Restaurant
1,East Toronto,"Brockton, Parkdale Village, Exhibition Place",43.679557,-79.352188,2,Café,Coffee Shop,Restaurant,Bar,Bakery,Furniture / Home Store,Gift Shop,Tibetan Restaurant,Thrift / Vintage Store,Performing Arts Venue
2,East Toronto,"Business reply mail Processing Centre, South C...",43.668999,-79.315572,0,Park,Pizza Place,Brewery,Coffee Shop,Fast Food Restaurant,Burrito Place,Sushi Restaurant,Bakery,Italian Restaurant,Electronics Store
3,East Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.659526,-79.340923,4,Café,Harbor / Marina,Coffee Shop,Garden,Airport,Sculpture Garden,Dog Run,Sushi Restaurant,Scenic Lookout,Park
4,Central Toronto,Central Bay Street,43.728020,-79.388790,0,Coffee Shop,Clothing Store,Ramen Restaurant,Park,Café,Gastropub,Furniture / Home Store,Sushi Restaurant,Japanese Restaurant,Juice Bar


In [159]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

In [161]:
### Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Business reply mail Processing Centre, South C...",Park,Pizza Place,Brewery,Coffee Shop,Fast Food Restaurant,Burrito Place,Sushi Restaurant,Bakery,Italian Restaurant,Electronics Store
4,Central Bay Street,Coffee Shop,Clothing Store,Ramen Restaurant,Park,Café,Gastropub,Furniture / Home Store,Sushi Restaurant,Japanese Restaurant,Juice Bar
5,Christie,Korean Restaurant,Café,Coffee Shop,Grocery Store,Cocktail Bar,Mexican Restaurant,Diner,Park,Ice Cream Shop,Pizza Place
6,Church and Wellesley,Coffee Shop,Park,Japanese Restaurant,Diner,Yoga Studio,Restaurant,Ramen Restaurant,Pizza Place,Men's Store,Ice Cream Shop
8,Davisville,Italian Restaurant,Coffee Shop,Sushi Restaurant,Café,Pizza Place,Restaurant,Dessert Shop,Fast Food Restaurant,Gym,Indian Restaurant
9,Davisville North,Coffee Shop,Italian Restaurant,Restaurant,Café,Pizza Place,Gym,Dessert Shop,Fast Food Restaurant,Sushi Restaurant,Movie Theater
10,"Dufferin, Dovercourt Village",Café,Coffee Shop,Park,Sushi Restaurant,Bar,Pharmacy,Portuguese Restaurant,Bakery,Italian Restaurant,Brewery
13,"Garden District, Ryerson",Coffee Shop,Gastropub,Japanese Restaurant,Restaurant,Ramen Restaurant,Italian Restaurant,Pizza Place,Cosmetics Shop,Creperie,Burrito Place
15,"High Park, The Junction South",Café,Bar,Coffee Shop,Thai Restaurant,Sushi Restaurant,Italian Restaurant,Convenience Store,Sandwich Place,Nail Salon,Gym
16,"India Bazaar, The Beaches West",Indian Restaurant,Coffee Shop,Beach,Park,Café,Harbor / Marina,Brewery,Bakery,Fast Food Restaurant,Burrito Place


In [162]:
### Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Café,Coffee Shop,Bookstore,College Quad,Gym / Fitness Center,Park,College Gym,Trail,Electronics Store,Doner Restaurant


In [163]:
### Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Park,Beer Bar,Bakery,Farmers Market,Seafood Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Restaurant,Bar,Bakery,Furniture / Home Store,Gift Shop,Tibetan Restaurant,Thrift / Vintage Store,Performing Arts Venue
7,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,Seafood Restaurant,Japanese Restaurant,Concert Hall,Cosmetics Shop,Italian Restaurant,Farmers Market
11,"First Canadian Place, Underground city",Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Theater,Concert Hall,Vegetarian / Vegan Restaurant,Gym,Bookstore
14,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Café,Hotel,Park,Gym,Japanese Restaurant,Theater,Brewery,Baseball Stadium,Scenic Lookout
19,"Little Portugal, Trinity",Café,Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Asian Restaurant,Cocktail Bar,Pizza Place,Italian Restaurant,Gift Shop
25,"Richmond, Adelaide, King",Coffee Shop,Café,Hotel,Theater,Restaurant,Movie Theater,Clothing Store,Beer Bar,Italian Restaurant,Japanese Restaurant
29,St. James Town,Café,Coffee Shop,Restaurant,Seafood Restaurant,Hotel,Japanese Restaurant,Cosmetics Shop,Plaza,Beer Bar,Pizza Place
31,Stn A PO Boxes,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Seafood Restaurant,Beer Bar,Gastropub,Bakery,Park
37,"Toronto Dominion Centre, Design Exchange",Café,Hotel,Coffee Shop,Japanese Restaurant,Restaurant,Concert Hall,Theater,Gym,Bookstore,Beer Bar


In [164]:
### Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"Forest Hill North & West, Forest Hill Road Park",Coffee Shop,Park,Café,Bank,Burger Joint,Skating Rink,Italian Restaurant,Japanese Restaurant,Gym / Fitness Center,Trail
20,"Moore Park, Summerhill East",Coffee Shop,Grocery Store,Italian Restaurant,Park,Gym,Sandwich Place,Thai Restaurant,Bagel Shop,Pizza Place,Pub
26,Rosedale,Coffee Shop,Park,Grocery Store,Convenience Store,Metro Station,Pie Shop,Breakfast Spot,Candy Store,Bistro,Sandwich Place
27,Roselawn,Sushi Restaurant,Italian Restaurant,Café,Coffee Shop,Bank,Pharmacy,Pet Store,Skating Rink,Gym Pool,Dance Studio
33,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Sushi Restaurant,Thai Restaurant,Italian Restaurant,Park,Grocery Store,Gym / Fitness Center,Liquor Store,Bagel Shop,Bank


In [165]:
### Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"CN Tower, King and Spadina, Railway Lands, Har...",Café,Harbor / Marina,Coffee Shop,Garden,Airport,Sculpture Garden,Dog Run,Sushi Restaurant,Scenic Lookout,Park
